In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,1270.44,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00
3,-0.00,299.59,-0.00,0.00,0.00,0.00,-0.00,-0.00
4,0.00,-0.00,63.17,-0.00,-0.00,0.00,-0.00,0.00
5,-0.00,0.00,-0.00,12.39,0.00,0.00,0.00,-0.00
6,-0.00,0.00,-0.00,0.00,2.43,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,0.52,-0.00,-0.00
8,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.15,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00003,-0.00004,-0.00002,-0.00005,0.00001
3,-0.00001,1.00000,-0.00000,0.00003,0.00001,0.00004,-0.00000,-0.00003
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00006,-0.00002,0.00003
5,-0.00003,0.00003,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00004
6,-0.00004,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00002,0.00004,0.00006,0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00005,-0.00000,-0.00002,0.00001,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.008849056862535

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.02006625e+05 1.09919873e+04 4.75275504e+02 1.75638817e+01
 6.34269254e-01 2.60398850e-02 1.78269751e-03 4.55039536e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999341,-0.035592,-0.007047,-0.001381,-0.000272,-0.000058,-0.000017,-0.000005
1,0.035297,0.998668,-0.036977,-0.006884,-0.001347,-0.000286,-0.000085,-0.000026
2,0.008277,0.036382,0.998431,-0.041005,-0.007682,-0.001628,-0.000482,-0.000147
3,0.001941,0.008227,0.040251,0.997787,-0.051109,-0.010394,-0.003053,-0.000939
4,0.000473,0.001998,0.009496,0.049758,0.995711,-0.074322,-0.020655,-0.006299
5,0.000129,0.000543,0.002576,0.013175,0.069276,0.984282,-0.156262,-0.042330
6,0.000053,0.000222,0.001054,0.005366,0.027438,0.134763,0.943312,-0.302023
7,0.000035,0.000149,0.000704,0.003594,0.018315,0.085987,0.292075,0.952339


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006594369497300301,
 0.0013321406787545342,
 0.0015689832625478406,
 0.0022133998992367454,
 0.004289420069483851,
 0.015718232791848474,
 0.056687533341385765,
 0.04766082787064563]